# Graph Traversal

Graphs are one of the most useful data structures known to us. They are networks consisting of vertices (or nodes) connected by edges (or arcs). They can be directed or undirected. Numerous kinds of problems can be represented as graph problems. These include problems in networking, logistics, mapping, and many other fields.

There's considerable literature on graph algorithms, which are an important part of discrete mathematics. These include algorithms for finding shortest paths, cycles (non-empty paths from a node to itself), and paths that reach all nodes (the famous "traveling salesman" problem, also known as Hamiltonian paths).

## Representing graphs

Few programming languages provide direct support for graphs as a data type, and Python is no exception. However, graphs can be easily represented using built-in types: dictionaries, sets, and lists.

<img src="images/graphex.png" width="400">

The graph above could be represented using a dictionary whose keys are the vertices and the associated values are lists or sets of vertices that have edges leading to them from the original vertex. This is called an **adjacency list** (or adjacency set) representation.

In [ ]:
graph_dict_with_lists = {
    'A': ['B', 'C'],
    'B': ['C', 'D'],
    'C': ['D'],
    'D': ['C'],
    'E': ['F'],
    'F': ['C'],
}

In [ ]:
graph_dict_with_sets = {
    'A': {'B', 'C'},
    'B': {'C', 'D'},
    'C': {'D'},
    'D': {'C'},
    'E': {'F'},
    'F': {'C'},
}

Another way to represent the graph would be with a square matrix (two-dimensional array), where each column and each row represent a vertex, and if there is an edge from the vertex in the given row to the vertex in the given column, then the value at that position is `1`, otherwise it's `0`. Here the first row and column correspond to vertex A, the second row and column correspond to vertex B, etc. Python doesn't have a separate matrix type, but we can use a list of lists.

In [ ]:
graph_matrix = [
    # To vertex
    #A  B  C  D  E  F    # From vertex
    [0, 1, 1, 0, 0, 0],  # A
    [0, 0, 1, 1, 0, 0],  # B
    [0, 0, 0, 1, 0, 0],  # C
    [0, 0, 1, 0, 0, 0],  # D
    [0, 0, 0, 0, 1, 0],  # E
    [0, 0, 1, 0, 0, 0],  # F
    
]

## Shortest Paths

One of the most common tasks in graphs is to find the shortest path (the path containing the fewest vertices) from one vertex to another. You had to solve this problem in Lab 2: these were the Bacon paths. The algorithm we saw there considered each neighbor of the starting vertex in one turn, then the neighbors of those nodes (if they haven't been visited yet) in the next turn, etc. This is called **breadth-first search**. It is illustrated by the animation below. (Image copyright: Blake Matheny, source: [Wikimedia Commons](https://en.wikipedia.org/wiki/File:Animated_BFS.gif).)

![Animated_BFS.gif](images/Animated_BFS.gif)

## A Dinner Invitation Problem

You have gotten into trouble with some of your, shall we say, difficult friends because they have realized that they are not being invited to your house parties. So you announce that you are going to have dinners on two consecutive nights, Friday and Saturday, and every one of your friends is going to get invited on exactly one of those two days. You are still worried about pairs of your friends who intensely dislike each other and want to invite them on different days.

To summarize:
1. Each of your friends must attend exactly one of the two dinners.
1. If A dislikes B or B dislikes A, they cannot both be in the same dinner party.

Now, you are a bit worried because you don’t know if you can pull this off.  If you had a small social circle like the one below, it would be easy. In the graph below, the vertices are friends and an edge between a pair of vertices mean that the two corresponding friends dislike each other, and cannot be invited to the party on the same night.

<img src="images/dinner1.png" width="400">

For Dinner 1 you could invite Bob, Don, and Cleo.  And for Dinner 2 you could invite Alice and Eve. There are other possibilities as well. Unfortunately, that social circle was from many moons ago and you have moved on. Your social circle now looks like this:

<img src="images/dinner2.png" width="400">

_Can you invite all of your friends A through I above following your self-imposed rules? Or are you going to have go to back on your word, and not invite someone?_

If you look at the graph carefully, you will see that you have three friends F, G and H, each of whom can’t be invited with other two.  You will need three separate days to host each of them. 😞

Okay, let’s assume that you convince G and H to not cause a ruckus if they see each other.  So your social graph now looks like this.

<img src="images/dinner3.png" width="400">

_What about now?_

### Finding a partition

With some work, you probably figured out that you could invite B, D, G, H and I to Dinner 1, and A, C, E and F to Dinner 2. Phew!

Your social circle is volatile and you might have to do this analysis week after week. You would like to quickly know if you can make an announcement any given week, such as “party on both nights, everyone gets to come to exactly one,” with the confidence that it will be a boisterous but friendly weekend. Which means you want to write a program that can check if there is some way that you can “partition,” i.e., split your friends across two nights, so if you look at the social circle graph, all the dislikes edges would be across the partition, and none within either side.

The partition we found looks like this:

<img src="images/dinner4.png" width="400">

This is simply redrawing your social circle graph! It turns out the graph above has a name – it is a **bipartite graph**. A bipartite graph is a graph whose vertices can be divided into two independent sets, U and V such that every edge _(u, v)_ either connects a vertex from U to V or a vertex from V to U. We can also say that there is no edge that connects vertices of same set.  In the above example U = {A, C, E, F} and V = {B, D, G, H, I}.

A graph is bipartite if the graph can be colored using two colors such that no two vertices that are adjacent, i.e., share an edge, have the same color. This is, of course, a restatement of our original problem, with colors substituted for dinner nights. We will sometimes refer to the coloring constraint as the _adjacency_ constraint.

You might think that a bipartite graph can’t have cycles. It is possible to color a graph with cycles comprising an _even_ number of vertices using two colors, let's say red and black. For example, see the graph below.

<img src="images/dinner5.png" width="400">

U = {A, C, E} and V = {B, D, F}, and we can invite members of U on one day, and members in V on the second. However, it is not possible to color a graph with two colors if has a cycle involving an odd number of vertices. Consider F, G and H in our second example with 9 vertices where G and H have an edge between them. F, G and H are in a 3-cycle and hence that graph is not bipartite. The 5-cycle in the graph below renders it non-bipartite.

<img src="images/dinner6.png" width="400">

Changing the color of A to black is not going to help. B and F need to be red, and then C and D need to be black, which violates an adjacency constraint.

### Checking if a Graph is Bipartite

We need an algorithm that given a graph successfully colors the vertices with two colors obeying the adjacency constraints if the graph is bipartite or tells us that the graph is not bipartite. One color will correspond to the set `U`, and the other to the set `V`. Here’s a possible algorithm that uses a technique called depth-first search.

1. Start with vertex `w` and `color = 'red'`.
1. If `w` has not yet been colored, color vertex `w` with `color`.
1. If `w` has already been colored with a different color from `color`, the graph is not bipartite, return `False`.
1. If `w` has already been colored with the correct color, return `True` and the (unmodified) coloring.
1. Flip `color`: `'red'` to `'black'`, or `'black'` to `'red'`.
1. For each neighbor `v` of `w`, recursively call the procedure with `v` and `color`, i.e., go to Step 2 with `w = v`. If any of the recursive calls return `False`, return `False`.
1. The graph is bipartite, return `True` and the coloring.

Let’s run this algorithm on an example graph shown below beginning with coloring vertex B Shaded.

<img src="images/coloring1.png" width="400">
<img src="images/coloring2.png" width="400">

C is the only vertex connected to B and is colored next. From C we go to D since B is already colored. Once we color D, since C is already colored, we have the choice of coloring E or F, and we color E first. Since E has no neighbors other than D, we next move to F, which is a neighbor of D. We color the neighbors of F in the order G, H and I.

An observant reader may have noticed that our example above omitted vertex/friend A from the social circle example we presented earlier. This is because A is not “connected” to any of the other vertices – it does not have any neighbors. Of course, A could be colored either red or black.

For simplicity, going forward We’ll assume that the input graph is such that we can reach all vertices from a specified start vertex.

Here’s the graph that we just executed the algorithm on in our chosen dictionary representation:

In [ ]:
graph = {'B': ['C'],
         'C': ['B', 'D'],
         'D': ['C', 'E', 'F'],
         'E': ['D'],
         'F': ['D', 'G', 'H', 'I'],
         'G': ['F'],
         'H': ['F'],
         'I': ['F']}

We have undirected graphs in this dinner invitation problem, meaning that each edge is undirected. This means that if we can traverse an edge from vertex B to vertex C, we can traverse the edge in the opposite direction from vertex C to vertex B. For our dictionary representation, this means that if a vertex key X has a vertex Y in its value list, then vertex key Y will also have vertex X in its value list. Check the dictionary `graph` above for this symmetry condition.

Now let's try to implement our algorithm in Python.

In [ ]:
def bipartite_color(graph, start, color, coloring):
    """Compute the bipartite coloring for a graph.

    Arguments:
        graph: the input graph represented as a dictionary.
        start: the vertex that will be colored first
        color: the color that will be applied to the starting vertex
        coloring: dictionary mapping vertices to colors

    Returns:
        bool: True if a bipartite coloring could be found
        dict: the coloring dictionary itself

    """
    pass

In [ ]:
bipartite_color(graph, 'B', 'red', {})

In [ ]:
original_graph = {'B': ['C'],
                  'C': ['B', 'D'],
                  'D': ['C', 'E', 'F'],
                  'E': ['D'],
                  'F': ['D', 'G', 'H', 'I'],
                  'G': ['F', 'H'],
                  'H': ['F', 'G'],
                  'I': ['F']}

In [ ]:
bipartite_color(original_graph, 'B', 'red', {})